Q1 and 2

1: B

2: D

In [ ]:
import numpy as np
import math

In [ ]:
num_trials = 1000
num_flips = 10
num_runs = 100000

epsilon = .3
e_out = .5
hoeffding_bound = 2 * np.exp(-2 * epsilon ** 2 * num_flips)

def coin_run():
  coin_flips = np.random.randint(2, size=(num_trials, num_flips))

  v_1 = coin_flips[0].sum() / num_flips

  c_rand = coin_flips[np.random.randint(num_trials)]
  v_rand = c_rand.sum() / num_flips

  heads_min = np.count_nonzero(coin_flips, axis=1)
  c_min = coin_flips[np.argmin(heads_min)]
  v_min = c_min.sum() / num_flips

  return v_1, v_rand, v_min

all_v = np.array([coin_run() for i in range(num_runs)])

In [ ]:
# Question 1
print(np.mean(all_v, axis=0)[-1])

0.037595999999976766


In [ ]:
# Question 2
def check_difference(x):
    return math.fabs(x - e_out) > epsilon

func = np.vectorize(check_difference)
print(np.mean(func(all_v), axis=0) <= hoeffding_bound)

[ True  True False]


Q5-10

In [ ]:
def random_point():
  return [np.random.uniform(-1, 1), np.random.uniform(-1, 1)]

# get corresponding y_n by evaluating target function on each x_n
def find_y(x, m, b):
  pos = m * x[0] - x[1] + b
  return np.sign(pos)

def generate_points(N):
  pt1 = random_point()
  pt2 = random_point()

  m = (pt2[1] - pt1[1]) / (pt2[0] - pt1[0]) # find slope of line
  b = pt1[1] - m * pt1[0]                   # find y-intercept of line

  X = []
  Y = []
  for i in range(N):
    new_pt = random_point()
    y = find_y(new_pt, m, b)
    X.append(new_pt)
    Y.append(y)

  return np.array(X), np.array(Y)

In [ ]:
MAX_ITER = 10000

def pla(X, Y, N, w):
  iterations = 0
  ones = np.ones((N, 1))
  temp_X = np.hstack((ones, X[:N]))

  while iterations < MAX_ITER:
    misclassified_pts = []

    for i in range(N):
      if np.sign(np.dot(w.T, temp_X[i])) != Y[i]:
        misclassified_pts.append(i)

    if not misclassified_pts:
      break

    random_idx = np.random.choice(misclassified_pts)
    w += Y[random_idx] * temp_X[random_idx]

    iterations += 1

  return iterations

In [ ]:
class LinearRegression:
  def __init__(self, X, Y, N):
    self.weights = np.zeros(X.shape[1])

    ones = np.ones((X.shape[0], 1))
    self.X = np.hstack((ones, X))
    self.Y = Y
    self.N = N

  def fit(self):
    temp_X = self.X[:self.N]

    X_transpose = np.transpose(temp_X)
    X_transpose_X = np.matmul(X_transpose, temp_X)
    X_inverse = np.linalg.inv(X_transpose_X)
    X_dagger = np.matmul(X_inverse, X_transpose)

    self.weights = np.matmul(X_dagger, self.Y[:self.N])

  def predict(self, X):
    return np.sign(np.matmul(X, self.weights))

  def calculate_error(self, f, g):
    return np.mean(f != g)

  def calculate_e_in(self):
    return self.calculate_error(self.predict(self.X[:N]), self.Y[:N])

  def calculate_e_out(self):
    return self.calculate_error(self.predict(self.X[N:]), self.Y[N:])

5: C

6: C

7: A

8: D

9: A

10: B

In [ ]:
# Problem 5 and 6
N = 100
runs = 1000

e_in = 0
e_out = 0

for i in range(runs):
  X, Y = generate_points(N + 1000)
  lin_reg = LinearRegression(X, Y, N)
  lin_reg.fit()
  e_in += lin_reg.calculate_e_in()
  e_out += lin_reg.calculate_e_out()

print(e_in / runs)
print(e_out / runs)

0.041200000000000125
0.04917799999999999


In [ ]:
# Problem 7
N = 10
avg_iter = 0

for i in range(runs):
  X, Y = generate_points(N)
  lin_reg = LinearRegression(X, Y, N)
  lin_reg.fit()

  cur_iter = pla(X, Y, N, lin_reg.weights)
  avg_iter += cur_iter

print(avg_iter / runs)

4.237


In [ ]:
def flip_for_noise(Y):
  n = len(Y)
  num_flips = int(0.1 * n)
  indices_to_flip = np.random.choice(n, num_flips, replace=False)
  Y[indices_to_flip] = -Y[indices_to_flip]

  return Y

def target_func(x_1, x_2):
  return np.sign(x_1 ** 2 + x_2 ** 2 - .6)

def generate_noisy_points(N):
  pt1 = random_point()
  pt2 = random_point()

  m = (pt2[1] - pt1[1]) / (pt2[0] - pt1[0]) # find slope of line
  b = pt1[1] - m * pt1[0]                   # find y-intercept of line

  X = []
  Y = []
  for i in range(N):
    new_pt = random_point()
    y = target_func(new_pt[0], new_pt[1])
    X.append(new_pt)
    Y.append(y)

  return np.array(X), flip_for_noise(np.array(Y))

In [ ]:
# Problem 8
N = 1000
e_in = 0

for i in range(runs):
  X, Y = generate_noisy_points(N)
  lin_reg = LinearRegression(X, Y, N)
  lin_reg.fit()
  e_in += lin_reg.calculate_e_in()

print(e_in / runs)

0.5036210000000005


In [ ]:
def transform_features(X):
    x1 = X[:, 0]
    x2 = X[:, 1]

    x1x2 = x1 * x2
    x1_squared = x1 ** 2
    x2_squared = x2 ** 2

    transform_X = np.column_stack((x1, x2, x1x2, x1_squared, x2_squared))

    return transform_X

In [ ]:
# Problem 9 and 10
avg_weights = np.zeros(6)
e_out = 0

for i in range(runs):
  X, Y = generate_noisy_points(N + 1000)
  X = transform_features(X)
  lin_reg = LinearRegression(X, Y, N)
  lin_reg.fit()
  avg_weights += lin_reg.weights
  e_out += lin_reg.calculate_e_out()

print(avg_weights / runs)
print(e_out / runs)

[-9.92119992e-01 -8.82924079e-04  6.85998494e-04 -8.51484226e-03
  1.55706379e+00  1.55667364e+00]
0.1262000000000001
